<a href="https://colab.research.google.com/github/chunbo777/twig_farm_nlp/blob/main/NLP_Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/chunbo777/NLP_text-style-transfer

Cloning into 'NLP_text-style-transfer'...
remote: Enumerating objects: 605, done.
remote: Counting objects: 100% (605/605), done.
remote: Compressing objects: 100% (242/242), done.
remote: Total 605 (delta 358), reused 598 (delta 355), pack-reused 0
Receiving objects: 100% (605/605), 37.00 MiB | 20.57 MiB/s, done.
Resolving deltas: 100% (358/358), done.


In [2]:
%cd /content/NLP_text-style-transfer

/content/NLP_text-style-transfer


In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 14.5 MB 5.4 MB/s 
     |████████████████████████████████| 10.5 MB 41.3 MB/s 
     |████████████████████████████████| 25.9 MB 1.7 MB/s 
     |████████████████████████████████| 1.1 MB 45.5 MB/s 
     |████████████████████████████████| 103 kB 74.0 MB/s 
     |████████████████████████████████| 2.6 MB 49.4 MB/s 
     |████████████████████████████████| 748.8 MB 15 kB/s 
     |████████████████████████████████| 69 kB 6.6 MB/s 
     |████████████████████████████████| 1.0 MB 39.7 MB/s 
     |████████████████████████████████| 320.6 MB 53 kB/s 
     |████████████████████████████████| 895 kB 28.4 MB/s 
     |████████████████████████████████| 3.0 MB 40.0 MB/s 
     |████████████████████████████████| 320.5 MB 14 kB/s 
     |████████████████████████████████| 320.4 MB 11 kB/s 
     |████████████████████████████████| 320.4 MB 22 kB/s 
     |████████████████████████████████| 320.4 MB 47 kB/s 
INFO: pip is looking at multiple versions of six to determine which versi

In [ ]:
import sys
sys.argv=['']
del sys

In [ ]:
#transfer.py

from tqdm import tqdm
import torch

from bert_pretrained import bert_tokenizer
from options import args


def style_transfer(encoder=None, generator=None, text_path=None, n_samples=100):
    # save result if path is given
    if args.transfer_result_save_path is not None:
        fw = open(args.transfer_result_save_path, 'a')
    else:
        fw = None

    # interactive mode
    if text_path is None:
        if fw is not None:
            fw.write('\n' + "=" * 50 + '\n')
            fw.write("Interactive transfer from {} -> {}\n".format(
                str(1 - args.transfer_to),
                str(args.transfer_to)
            ))
            fw.write("=" * 50 + '\n')
        try:
            while True:
                fmt = "Enter text to transfer to style {} (Ctrl+C to exit): "
                text = input(fmt.format(args.transfer_to))
                tokens = bert_tokenizer.encode(text, add_special_tokens=False)
                tokens = (
                    [bert_tokenizer.bos_token_id]
                    + tokens
                    + [bert_tokenizer.eos_token_id]
                )
                tokens = torch.LongTensor([tokens]).transpose(0, 1)
                original_label = torch.FloatTensor([1 - args.transfer_to])
                output = generate_text(
                    encoder.to(args.device),
                    generator.to(args.device),
                    original_label.to(args.device),
                    tokens.to(args.device)
                )
                print("Transfer result:", output)
                if fw is not None:
                    fw.write(text + ' -> ' + output + '\n')

        except KeyboardInterrupt:
            if fw is not None:
                fw.close()
            print("\nEnd interactive transfer\n")

    # load data from text path
    else:
        if fw is not None:
            fw.write('\n' + "=" * 50 + '\n')
            fw.write("Transfer from file: {}\n".format(text_path))
            fw.write("Number of samples: {}\n".format(n_samples))
            fw.write("=" * 50 + '\n')

        pbar = tqdm(total=n_samples)
        counter = 0
        inputs0, inputs1 = [], []
        outputs0, outputs1 = [], []
        with open(text_path, 'r') as text_file:
            for line in text_file:
                counter += 1
                if counter == 1:
                    continue
                _, text, label = line.strip().split('\t')
                tokens = bert_tokenizer.encode(text, add_special_tokens=False)
                tokens = (
                    [bert_tokenizer.bos_token_id]
                    + tokens
                    + [bert_tokenizer.eos_token_id]
                )
                tokens = torch.LongTensor([tokens]).transpose(0, 1)
                original_label = torch.FloatTensor([int(label)])
                output = generate_text(
                    encoder.to(args.device),
                    generator.to(args.device),
                    original_label.to(args.device),
                    tokens.to(args.device)
                )
                if int(label) == 0:
                    inputs0.append(text)
                    outputs0.append(output)
                else:
                    inputs1.append(text)
                    outputs1.append(output)
                pbar.update()
                if fw is not None:
                    fw.write(label + ' > ' + str(1-int(label)) + ': '+ text + ' -> ' + output + '\n')
                if counter > n_samples:
                    break

        if fw is not None:
            fw.close()
        return inputs0, inputs1, outputs0, outputs1


def generate_text(encoder, generator, original_label, tokens):
    src_len = [len(tokens)]
    predictions = generator.transfer(
        encoder(original_label, tokens, src_len),  # hidden state
        1 - original_label,  # transfer label
        eos_token_id=bert_tokenizer.eos_token_id,
        max_len=args.transfer_max_len
    )
    # change this part to first occurence of ber_tokenizer.eos_token_id
    #if predictions[-1] == bert_tokenizer.eos_token_id:
    #    predictions = predictions[:-1]

    try:
        eos_idx = predictions.index(bert_tokenizer.eos_token_id)
        predictions = predictions[:eos_idx]
    except ValueError:
        predictions=predictions[:-1]
    return bert_tokenizer.decode(predictions)


def _transfer():
    #cuda device 설정
    device = torch.device('cuda:{}'.format(args.cuda_device) if torch.cuda.is_available() else 'cpu')
    
    # 1. get model
    embedding = get_bert_word_embedding().to(device).eval()
    encoder = Encoder(embedding, args.dim_y, args.dim_z).to(device).eval()
    generator = Generator(embedding, args.dim_y, args.dim_z, args.temperature, bert_tokenizer.bos_token_id, use_gumbel=args.use_gumbel).to(device).eval()
    
    # 2. load checkpoint
    ckpt = torch.load(args.ckpt_path, map_location=device)
    embedding.load_state_dict(ckpt['embedding_state_dict'])
    encoder.load_state_dict(ckpt['encoder_state_dict'])
    generator.load_state_dict(ckpt['generator_state_dict'])
    
    # 3. transfer!
    if args.transfer_result_save_path is not None:
        fw = open(args.transfer_result_save_path, 'w')
    else:
        fw = None
            
    if args.test_text_path is None:
        # interactive mode
        while True:
            text = input("Enter text to transfer to stye {} (Ctrl+C to exit): ".format(args.transfer_to))
            text_tokens = [bert_tokenizer.bos_token_id] + bert_tokenizer.encode(text, add_special_tokens=False) + [bert_tokenizer.eos_token_id]
            #텍스트 토큰 만들기 bos+encoding+eos
            text_tokens_tensor = torch.LongTensor([text_tokens]).transpose(0, 1).to(device)
            #long tensor transpose
            src_len = [len(text_tokens)]
            original_label = torch.FloatTensor([1-args.transfer_to]).to(device)
            transfer_label = torch.FloatTensor([args.transfer_to]).to(device)
            
            z = encoder(original_label, text_tokens_tensor, src_len)
            predictions = generator.transfer(z, transfer_label, eos_token_id=bert_tokenizer.eos_token_id, max_len=args.transfer_max_len)
            if predictions[-1] == bert_tokenizer.eos_token_id:
                predictions = predictions[:-1]
                
            result = bert_tokenizer.decode(predictions)
            print("Transfer Result:", result)
            if fw is not None:
                fw.write(text + ' -> ' + result + '\n')
                
            if args.test_recon:
                recon = generator.transfer(z, original_label, eos_token_id=bert_tokenizer.eos_token_id, max_len=args.transfer_max_len)
                if recon[-1] == bert_tokenizer.eos_token_id:
                    recon = recon[:-1]
                print("Recon:", bert_tokenizer.decode(recon))
            
    else:

        for line in args.test_text_path:
            line = line.strip()
            text = line
            text_tokens = [bert_tokenizer.bos_token_id] + bert_tokenizer.encode(text, add_special_tokens=False) + [bert_tokenizer.eos_token_id]
            text_tokens_tensor = torch.LongTensor([text_tokens]).transpose(0, 1).to(device)
            src_len = [len(text_tokens)]
            original_label = torch.FloatTensor([1-args.transfer_to]).to(device)
            transfer_label = torch.FloatTensor([args.transfer_to]).to(device)
            
            z = encoder(original_label, text_tokens_tensor, src_len)
            predictions = generator.transfer(z, transfer_label, eos_token_id=bert_tokenizer.eos_token_id, max_len=args.transfer_max_len)
            if predictions[-1] == bert_tokenizer.eos_token_id:
                predictions = predictions[:-1]
                
            result = bert_tokenizer.decode(predictions)
            print("Transfer Result:", result)
            if fw is not None:
                fw.write(text + ' -> ' + result + '\n')
                
            if args.test_recon:
                recon = generator.transfer(z, original_label, eos_token_id=bert_tokenizer.eos_token_id, max_len=args.transfer_max_len)
                if recon[-1] == bert_tokenizer.eos_token_id:
                    recon = recon[:-1]
                print("Recon:", bert_tokenizer.decode(recon))
            
if __name__ == '__main__':
    transfer()


usage:  [-h] --ckpt_path CKPT_PATH [--clf_ckpt_path CLF_CKPT_PATH]
        [--dataset {yelp,nsmc}] [--text_file_path TEXT_FILE_PATH]
        [--val_text_file_path VAL_TEXT_FILE_PATH] [--batch_size BATCH_SIZE]
        [--max_seq_length MAX_SEQ_LENGTH] [--num_workers NUM_WORKERS]
        [--dim_y DIM_Y] [--dim_z DIM_Z] [--dim_emb DIM_EMB]
        [--filter_sizes FILTER_SIZES [FILTER_SIZES ...]]
        [--n_filters N_FILTERS] [--epochs EPOCHS]
        [--weight_decay WEIGHT_DECAY] [--max_grad_norm MAX_GRAD_NORM]
        [--lr LR] [--disc_lr DISC_LR] [--temperature TEMPERATURE]
        [--use_gumbel USE_GUMBEL] [--rho RHO] [--two_stage TWO_STAGE]
        [--second_stage_num SECOND_STAGE_NUM]
        [--gan_type {vanilla,wgan-gp,lsgan}] [--gp_weight GP_WEIGHT]
        [--log_interval LOG_INTERVAL] [--language {ko,en}]
        [--threshold THRESHOLD] [--mode {train,transfer,interactive}]
        [--test_text_path TEST_TEXT_PATH] [--transfer_to {0,1}]
        [--n_samples N_SAMPLES] [--trans

SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#train.py

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
import time
import itertools
import sys, os

from dataloader import get_dataloader_for_style_transfer
from model import Encoder, Generator, Discriminator
from bert_pretrained import bert_tokenizer, get_bert_word_embedding, FILE_ID
from bert_pretrained.classifier import BertClassifier
from loss import loss_fn, gradient_penalty
from evaluate import calculate_accuracy, calculate_frechet_distance
from transfer import style_transfer

from options import args
from utils import AverageMeter, ProgressMeter, download_google, Metric_Printer


class Trainer:
    def __init__(self):
        # get models
        embedding = get_bert_word_embedding()
        if os.path.isfile(args.load_ckpt):
            self.models = torch.load(args.load_ckpt)
        else:
            self.models = nn.ModuleDict({
                'embedding': embedding,
                'encoder': Encoder(embedding, args.dim_y, args.dim_z),
                'generator': Generator(
                    embedding, args.dim_y, args.dim_z, args.temperature,
                    bert_tokenizer.bos_token_id, use_gumbel=args.use_gumbel
                ),
                'disc_0': Discriminator(  # 0: real, 1: fake
                    args.dim_y + args.dim_z, args.n_filters, args.filter_sizes
                ),
                'disc_1': Discriminator(  # 1: real, 0: fake
                    args.dim_y + args.dim_z, args.n_filters, args.filter_sizes
                ),
            })
        self.models.to(args.device)
        # pretrained classifier
        self.clf = BertClassifier()
        if args.clf_ckpt_path is not None:
            download_google(FILE_ID, args.clf_ckpt_path)
            ckpt = torch.load(
                args.clf_ckpt_path,
                map_location=lambda storage, loc: storage
            )
            self.clf.load_state_dict(ckpt['model_state_dict'])
        self.clf.to(args.device)
        self.clf.eval()

        # get dataloaders
        self.train_loaders = get_dataloader_for_style_transfer(
            args.text_file_path, shuffle=True, drop_last=True
        )
        # label placeholders
        self.zeros = torch.zeros(args.batch_size, 1).to(args.device)
        self.ones = torch.ones(args.batch_size, 1).to(args.device)

        # get optimizers
        self.optimizer = optim.AdamW(
            list(itertools.chain.from_iterable([
                list(self.models[k].parameters())
                for k in ['embedding', 'encoder', 'generator']
            ])),
            lr=args.lr,
            betas=(0.5, 0.9),
            weight_decay=args.weight_decay
        )
        self.disc_optimizer = optim.AdamW(
            list(itertools.chain.from_iterable([
                list(self.models[k].parameters())
                for k in ['disc_0', 'disc_1']
            ])),
            lr=args.disc_lr,
            betas=(0.5, 0.9),
            weight_decay=args.weight_decay
        )

        self.epoch = 0

    def train_epoch(self):
        self.models.train()
        self.epoch += 1

        # record training statistics
        avg_meters = {
            'loss_rec': AverageMeter('Loss Rec', ':.4e'),
            'loss_adv': AverageMeter('Loss Adv', ':.4e'),
            'loss_disc': AverageMeter('Loss Disc', ':.4e'),
            'time': AverageMeter('Time', ':6.3f')
        }
        progress_meter = ProgressMeter(
            len(self.train_loaders[0]),
            avg_meters.values(),
            prefix="Epoch: [{}]".format(self.epoch)
        )

        # begin training from minibatches
        for ix, (data_0, data_1) in enumerate(zip(*self.train_loaders)):
            start_time = time.time()

            # load text and labels
            src_0, src_len_0, labels_0 = data_0
            src_0, labels_0 = src_0.to(args.device), labels_0.to(args.device)
            src_1, src_len_1, labels_1 = data_1
            src_1, labels_1 = src_1.to(args.device), labels_1.to(args.device)

            # encode
            encoder = self.models['encoder']
            z_0 = encoder(labels_0, src_0, src_len_0)  # (batch_size, dim_z)
            z_1 = encoder(labels_1, src_1, src_len_1)

            # recon & transfer
            generator = self.models['generator']
            inputs_0 = (z_0, labels_0, src_0)
            h_ori_seq_0, pred_ori_0 = generator(*inputs_0, src_len_0, False)
            h_trans_seq_0_to_1, _ = generator(*inputs_0, src_len_1, True)

            inputs_1 = (z_1, labels_1, src_1)
            h_ori_seq_1, pred_ori_1 = generator(*inputs_1, src_len_1, False)
            h_trans_seq_1_to_0, _ = generator(*inputs_1, src_len_0, True)

            # discriminate real and transfer
            disc_0, disc_1 = self.models['disc_0'], self.models['disc_1']
            d_0_real = disc_0(h_ori_seq_0.detach())  # detached
            d_0_fake = disc_0(h_trans_seq_1_to_0.detach())
            d_1_real = disc_1(h_ori_seq_1.detach())
            d_1_fake = disc_1(h_trans_seq_0_to_1.detach())

            # discriminator loss
            loss_disc = (
                loss_fn(args.gan_type)(d_0_real, self.ones)
                + loss_fn(args.gan_type)(d_0_fake, self.zeros)
                + loss_fn(args.gan_type)(d_1_real, self.ones)
                + loss_fn(args.gan_type)(d_1_fake, self.zeros)
            )
            # gradient penalty
            if args.gan_type == 'wgan-gp':
                loss_disc += args.gp_weight * gradient_penalty(
                    h_ori_seq_0,            # real data for 0
                    h_trans_seq_1_to_0,     # fake data for 0
                    disc_0
                )
                loss_disc += args.gp_weight * gradient_penalty(
                    h_ori_seq_1,            # real data for 1
                    h_trans_seq_0_to_1,     # fake data for 1
                    disc_1
                )
            avg_meters['loss_disc'].update(loss_disc.item(), src_0.size(0))

            self.disc_optimizer.zero_grad()
            loss_disc.backward()
            self.disc_optimizer.step()

            # reconstruction loss
            loss_rec = (
                F.cross_entropy(    # Recon 0 -> 0
                    pred_ori_0.view(-1, pred_ori_0.size(-1)),
                    src_0[1:].view(-1),
                    ignore_index=bert_tokenizer.pad_token_id,
                    reduction='sum'
                )
                + F.cross_entropy(  # Recon 1 -> 1
                    pred_ori_1.view(-1, pred_ori_1.size(-1)),
                    src_1[1:].view(-1),
                    ignore_index=bert_tokenizer.pad_token_id,
                    reduction='sum'
                )
            ) / (2.0 * args.batch_size)  # match scale with the orginal paper
            avg_meters['loss_rec'].update(loss_rec.item(), src_0.size(0))

            # generator loss
            d_0_fake = disc_0(h_trans_seq_1_to_0)  # not detached
            d_1_fake = disc_1(h_trans_seq_0_to_1)
            loss_adv = (
                loss_fn(args.gan_type, disc=False)(d_0_fake, self.ones)
                + loss_fn(args.gan_type, disc=False)(d_1_fake, self.ones)
            ) / 2.0  # match scale with the original paper
            avg_meters['loss_adv'].update(loss_adv.item(), src_0.size(0))

            # XXX: threshold for training stability
            if (not args.two_stage):
                if (args.threshold is not None
                        and loss_disc < args.threshold):
                    loss = loss_rec + args.rho * loss_adv
                else:
                    loss = loss_rec
            else: # two_stage training
                if (args.second_stage_num > args.epochs-self.epoch): 
                    # last second_stage; flow loss_adv gradients
                    loss = loss_rec + args.rho * loss_adv
                else:
                    loss = loss_rec
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            avg_meters['time'].update(time.time() - start_time)

            # log progress
            if (ix + 1) % args.log_interval == 0:
                progress_meter.display(ix + 1)

        progress_meter.display(len(self.train_loaders[0]))

    def evaluate(self):
        self.models.eval()
        # generate samples
        inputs0, inputs1, outputs0, outputs1 = style_transfer(
            encoder=self.models['encoder'],
            generator=self.models['generator'],
            text_path=args.val_text_file_path,
            n_samples=args.n_samples
        )

        # display 10 samples for each
        print('=' * 30 + '\nnegative -> positive\n' + '=' * 30 + '\n')
        for original, transfer in zip(inputs0[:10], outputs0[:10]):
            print(original + ' -> ' + transfer + '\n')
        print('=' * 30 + '\npositive -> negative\n' + '=' * 30 + '\n')
        for original, transfer in zip(inputs1[:10], outputs1[:10]):
            print(original + ' -> ' + transfer + '\n')

        print("Evaluation from {} samples".format(args.n_samples))
        fed = (calculate_frechet_distance(inputs1, outputs0)
               + calculate_frechet_distance(inputs0, outputs1))
        print('FED: {:.4f}'.format(fed))

        loss, acc = calculate_accuracy(
            self.clf,
            outputs0 + outputs1,
            torch.cat([
                torch.ones(len(outputs0)),
                torch.zeros(len(outputs1))
            ]).long().to(args.device)
        )
        print('Loss: {:.4f}'.format(loss.item()))
        print('Accuracy: {:.4f}\n'.format(acc.item()))
        return fed, loss.item(), acc.item()


class Translator:
    def __init__(self):
        self.models = torch.load(args.ckpt_path)
    def transfer(self):
        self.models.eval()
        if args.mode == 'interactive':
            args.test_text_path = None
        _, _, _, _ = style_transfer(
            encoder=self.models['encoder'],
            generator=self.models['generator'],
            text_path=args.test_text_path,
            n_samples=args.n_samples
        )


if __name__ == '__main__':
    if args.mode == 'train':
        trainer = Trainer()
        printer = Metric_Printer('FED', 'Loss', 'Acc')
        
        loss_save = sys.maxsize
        for _ in range(args.epochs):

            trainer.train_epoch()
            fed, loss, acc = trainer.evaluate()
            if loss < loss_save:
                loss_save = loss
                print ("saving model : " + args.ckpt_path)
                torch.save(trainer.models, args.ckpt_path)

            printer.update(fed, loss, acc)
        print(printer)
    else:
        translator = Translator()
        translator.transfer()

In [ ]:
!python train.py

In [ ]:
pwd

In [ ]:
%cd mkdir tmp

In [ ]:
#options.py

import sys
import argparse
import pprint
import torch
from utils import str2bool


argparser = argparse.ArgumentParser(sys.argv[0])

argparser.add_argument('--ckpt_path',
                       required=True,
                       help="path to save/load checkpoint",
                       type=str,
                       default="/content/NLP_text-style-transfer/tmp")
argparser.add_argument('--clf_ckpt_path',
                       help="path to load pretrained classifier",
                       type=str,
                       default="/content/NLP_text-style-transfer/bert_pretrained/classifier.py")

# dataloading
argparser.add_argument('--dataset',
                       type=str,
                       choices=['yelp', 'nsmc'],
                       default=None)
argparser.add_argument('--text_file_path',
                       type=str)
argparser.add_argument('--val_text_file_path',
                       type=str)
argparser.add_argument('--batch_size',
                       type=int,
                       default=64)
argparser.add_argument('--max_seq_length',
                       type=int,
                       default=64)
argparser.add_argument('--num_workers',
                       type=int,
                       default=4)

# architecture
argparser.add_argument('--dim_y',
                       type=int,
                       default=200)
argparser.add_argument('--dim_z',
                       type=int,
                       default=500)
argparser.add_argument('--dim_emb',
                       type=int,
                       default=768)
argparser.add_argument('--filter_sizes',
                       type=int,
                       nargs='+',
                       default=[1, 2, 3, 4, 5])
argparser.add_argument('--n_filters',
                       type=int,
                       default=128)

# learning
argparser.add_argument('--epochs',
                       type=int,
                       default=20)
argparser.add_argument('--weight_decay',
                       type=float,
                       default=0.0)
argparser.add_argument('--max_grad_norm',
                       type=float,
                       default=1.0)
argparser.add_argument('--lr',
                       type=float,
                       default=5e-4)
argparser.add_argument('--disc_lr',
                       type=float,
                       default=5e-5)
argparser.add_argument("--temperature",
                       type=float,
                       default=0.1)
argparser.add_argument('--use_gumbel',
                       default=True,
                       type=str2bool)
argparser.add_argument('--rho',  # loss_rec + rho * loss_adv
                       type=float,
                       default=1)
argparser.add_argument('--two_stage',
                        type=str2bool,
                        default=False)
argparser.add_argument('--second_stage_num',
                        type=int,
                        default=2)
argparser.add_argument('--gan_type',
                       default='vanilla',
                       choices=['vanilla', 'wgan-gp', 'lsgan'])
argparser.add_argument('--gp_weight',
                       default=1.0,
                       type=float)
argparser.add_argument('--log_interval',
                       default=100,
                       type=int)
argparser.add_argument('--language',
                       default='ko',
                       choices=['ko', 'en'])
argparser.add_argument('--threshold',
                       type=float,
                       default=None)

# testing
argparser.add_argument('--mode',
                        help='train or transfer',
                        choices=['train', 'transfer', 'interactive'],
                        default='train',
                        type=str)
argparser.add_argument('--test_text_path',
                       help='path to text file whose each line contains one sentence',
                       default=None)
argparser.add_argument('--transfer_to', #only use in interactive transfer
                       default=1,
                       type=int,
                       choices=[0, 1])
argparser.add_argument('--n_samples',
                       default=1000,
                       type=int)
argparser.add_argument('--transfer_max_len',
                       default=64,
                       type=int)
argparser.add_argument("--transfer_result_save_path",
                       default=None,
                       help="path to save transfer result")

# others
argparser.add_argument("--cuda_device",
                       type=int,
                       default=0)
argparser.add_argument("--load_ckpt", # Tobe deleted after experimenting
                       type=str,
                       default="ckpts/wgan_no_threshold.pt")

args = argparser.parse_args()

args.device = torch.device(
    'cuda:{}'.format(args.cuda_device) if torch.cuda.is_available() else 'cpu'
)

# dataset presets
if args.dataset == 'yelp':
    args.language = 'en'
    args.text_file_path = 'data/yelp/yelp.sentiment.train'
    args.val_text_file_path = 'data/yelp/yelp.sentiment.val'
    args.test_text_path = 'data/yelp/yelp.sentiment.test'
    args.clf_ckpt_path = 'checkpoints/yelp_clf.pt'
elif args.dataset == 'nsmc':
    args.language = 'ko'
    args.text_file_path = 'data/nsmc/ratings_train.txt'
    args.val_text_file_path = 'data/nsmc/ratings_test.txt'
    args.test_text_path = 'data/nsmc/ratings_test.txt'
    args.clf_ckpt_path = 'checkpoints/nsmc_clf.pt'
elif args.dataset is None:
    assert args.text_file_path is not None
    assert args.val_text_file_path is not None
    assert args.test_text_path is not None

print('------------------------------------------------')
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(vars(args))
print('------------------------------------------------')


In [ ]:
%cd /content/NLP_text-style-transfer


/content/NLP_text-style-transfer


In [ ]:
%mkdir checkpoint

In [ ]:
!python train.py --ckpt_path "checkpoints" --clf_ckpt_path "checkpoints" --dataset yelp --text_file_path "data" --val_text_file_path "data" --gan_type vanilla --language en --threshold 0.1 --mode interactive --test_text_path "data" --transfer_to 0
       

------------------------------------------------
{   'batch_size': 64,
    'ckpt_path': 'checkpoints',
    'clf_ckpt_path': 'checkpoints/yelp_clf.pt',
    'cuda_device': 0,
    'dataset': 'yelp',
    'device': device(type='cpu'),
    'dim_emb': 768,
    'dim_y': 200,
    'dim_z': 500,
    'disc_lr': 5e-05,
    'epochs': 20,
    'filter_sizes': [1, 2, 3, 4, 5],
    'gan_type': 'vanilla',
    'gp_weight': 1.0,
    'language': 'en',
    'load_ckpt': 'ckpts/wgan_no_threshold.pt',
    'log_interval': 100,
    'lr': 0.0005,
    'max_grad_norm': 1.0,
    'max_seq_length': 64,
    'mode': 'interactive',
    'n_filters': 128,
    'n_samples': 1000,
    'num_workers': 4,
    'rho': 1,
    'second_stage_num': 2,
    'temperature': 0.1,
    'test_text_path': 'data/yelp/yelp.sentiment.test',
    'text_file_path': 'data/yelp/yelp.sentiment.train',
    'threshold': 0.1,
    'transfer_max_len': 64,
    'transfer_result_save_path': None,
    'transfer_to': 0,
    'two_stage': False,
    'use_gumbel': T

In [ ]:
#utils.py

import os
import requests
import torch
from torch.autograd import Function
import numpy as np
import scipy.linalg

URL = "https://docs.google.com/uc?export=download"


def save_model(model, ckpt_path):
    pardir = os.path.dirname(os.path.abspath(ckpt_path))
    if not os.path.isdir(pardir):
        os.makedirs(pardir, exist_ok=True)
    torch.save(
        {'model_state_dict': model.state_dict()},
        ckpt_path
    )


def download_google(file_id, filename):
    pardir = os.path.dirname(os.path.abspath(filename))
    if not os.path.isdir(pardir):
        os.makedirs(pardir, exist_ok=True)
    if not os.path.isfile(filename):
        # Request file from URL
        session = requests.Session()
        response = session.get(URL, params={'id': file_id}, stream=True)
        for key, value in response.cookies.items():
            if key.startswith('download_warning'):
                params = {'id': file_id, 'confirm': value}
                response = session.get(URL, params=params, stream=True)

        # Download file
        with open(filename, 'wb') as f:
            f.write(response.content)


def str2bool(s):
    if s.lower() in ['true', 't', 'yes']:
        return True
    return False


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'


def covariance(m, rowvar=True, inplace=False):
    '''Estimate a covariance matrix given data.

    Covariance indicates the level to which two variables vary together.
    If we examine N-dimensional samples, `X = [x_1, x_2, ... x_N]^T`,
    then the covariance matrix element `C_{ij}` is the covariance of
    `x_i` and `x_j`. The element `C_{ii}` is the variance of `x_i`.

    Args:
        m: A 1-D or 2-D array containing multiple variables and observations.
            Each row of `m` represents a variable, and each column a single
            observation of all those variables.
        rowvar: If `rowvar` is True, then each row represents a
            variable, with observations in the columns. Otherwise, the
            relationship is transposed: each column represents a variable,
            while the rows contain observations.

    Returns:
        The covariance matrix of the variables.
    '''
    if m.dim() > 2:
        raise ValueError('m has more than 2 dimensions')
    if m.dim() < 2:
        m = m.view(1, -1)
    if not rowvar and m.size(0) != 1:
        m = m.t()
    # m = m.type(torch.double)  # uncomment this line if desired
    fact = 1.0 / (m.size(1) - 1)
    if inplace:
        m -= torch.mean(m, dim=1, keepdim=True)
    else:
        m = m - torch.mean(m, dim=1, keepdim=True)
    mt = m.t()  # if complex: mt = m.t().conj()
    return fact * m.matmul(mt).squeeze()


class MatrixSquareRoot(Function):
    """Square root of a positive definite matrix.
    NOTE: matrix square root is not differentiable for matrices with
          zero eigenvalues.
    """
    @staticmethod
    def forward(ctx, input):
        m = input.detach().cpu().numpy().astype(np.float_)
        sqrtm = torch.from_numpy(scipy.linalg.sqrtm(m).real).to(input)
        ctx.save_for_backward(sqrtm)
        return sqrtm

    @staticmethod
    def backward(ctx, grad_output):
        grad_input = None
        if ctx.needs_input_grad[0]:
            sqrtm, = ctx.saved_tensors
            sqrtm = sqrtm.data.cpu().numpy().astype(np.float_)
            gm = grad_output.data.cpu().numpy().astype(np.float_)

            # Given a positive semi-definite matrix X,
            # since X = X^{1/2}X^{1/2}, we can compute the gradient of the
            # matrix square root dX^{1/2} by solving the Sylvester equation:
            # dX = (d(X^{1/2})X^{1/2} + X^{1/2}(dX^{1/2}).
            grad_sqrtm = scipy.linalg.solve_sylvester(sqrtm, sqrtm, gm)

            grad_input = torch.from_numpy(grad_sqrtm).to(grad_output)
        return grad_input


def sqrtm(m):
    return MatrixSquareRoot.apply(m)

class Metric_Printer(object):
    def __init__(self, *meters):
        self.meters = list(meters)
        self.values = []
    def update(self, *values):
        self.values.append(values)
    def __str__(self):
        names = '{:^6}'.format('epoch')
        val = ''
        for i in range(len(self.meters)):
            names += '{:^10}'.format(self.meters[i]) 
        names += '\n'
        for i in range(len(self.values)):
            val += '[{:^6}]'.format(i+1)
            for j in range(len(self.values[i])):
                val += '{:^10.4f}'.format(self.values[i][j])
            val += '\n' 
        return names + val

In [ ]:
#dataloader.py

import torch
from torch.utils.data import Dataset, DataLoader

from bert_pretrained.tokenizer import bert_tokenizer
from options import args


class SentimentAnalysis(Dataset):
    def __init__(self, txt_path, maxlen=256):
        self.maxlen = maxlen
        self.texts = []
        self.labels = []
        with open(txt_path) as fr:
            fr.readline()  # header line
            for line in fr:
                line = line.strip().split('\t')  # expects tsv format
                self.texts.append(line[1])
                self.labels.append(int(line[2]))

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        labels = self.labels[idx]
        # add [CLS] and [SEP] tokens
        text_tokens = bert_tokenizer.encode(text, add_special_tokens=True)
        text_tokens = (text_tokens[:min(len(text_tokens), self.maxlen) - 1]
                       + text_tokens[-1:])  # must always include [SEP]
        return torch.LongTensor(text_tokens), labels


class StyleTransfer(Dataset):
    def __init__(self, txt_path, maxlen=256, label=1):
        assert (bert_tokenizer.bos_token_id is not None
                and bert_tokenizer.eos_token_id is not None)
        self.maxlen = maxlen
        self.texts = []
        self.labels = []
        with open(txt_path) as fr:
            fr.readline()  # header line
            for line in fr:
                line = line.strip().split('\t')  # expects tsv format
                if int(line[2]) == label:
                    self.texts.append(line[1])
                    self.labels.append(int(line[2]))

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        labels = self.labels[idx]
        # exclude [CLS] and [SEP]
        text_tokens = bert_tokenizer.encode(text, add_special_tokens=False)
        text_tokens = ([bert_tokenizer.bos_token_id]
                       + text_tokens[:self.maxlen - 2]
                       + [bert_tokenizer.eos_token_id])
        return torch.LongTensor(text_tokens), labels


def get_dataloader_for_classification(txt_path, shuffle=True, drop_last=True):
    def collate_fn(data):
        input_ids, class_labels = zip(*data)

        input_ids_lens = [len(inp) for inp in input_ids]
        input_ids_maxlen = max(input_ids_lens)

        padded_input_ids = torch.zeros(len(input_ids), input_ids_maxlen, dtype=int)  # Long Tensor with (batch_size, maxlen)
        attention_mask = torch.zeros(len(input_ids), input_ids_maxlen, dtype=int)  # Long Tensor with (batch_size, maxlen), 1 if not padded 0 if padded

        for ix in range(len(input_ids)):
            padded_input_ids[ix, :input_ids_lens[ix]] = input_ids[ix]
            attention_mask[ix, :input_ids_lens[ix]] = 1

        class_labels = torch.LongTensor(class_labels)  # (batch_size, num_classes)

        return padded_input_ids, attention_mask, class_labels

    return DataLoader(
        SentimentAnalysis(txt_path, maxlen=args.max_seq_length),
        batch_size=args.batch_size,
        shuffle=shuffle,
        num_workers=args.num_workers,
        drop_last=drop_last,
        collate_fn=collate_fn
    )


def get_dataloader_for_style_transfer(txt_path, shuffle=True, drop_last=True):
    def collate_fn(data):
        input_ids, class_labels = zip(*data)

        input_ids_lens = [len(inp) for inp in input_ids]
        input_ids_maxlen = max(input_ids_lens)
        # Long Tensor with (maxlen, batch_size)
        padded_input_ids = torch.zeros(input_ids_maxlen, len(input_ids), dtype=int)
        padded_input_ids = padded_input_ids.fill_(bert_tokenizer.pad_token_id)

        for ix in range(len(input_ids)):
            padded_input_ids[:input_ids_lens[ix], ix] = input_ids[ix]

        class_labels = torch.FloatTensor(class_labels)  # (batch_size, num_classes)

        return padded_input_ids, input_ids_lens, class_labels

    dataloader0 = DataLoader(
        StyleTransfer(txt_path, maxlen=args.max_seq_length, label=0),
        batch_size=args.batch_size,
        shuffle=shuffle,
        num_workers=args.num_workers,
        drop_last=drop_last,
        collate_fn=collate_fn
    )
    dataloader1 = DataLoader(
        StyleTransfer(txt_path, maxlen=args.max_seq_length, label=1),
        batch_size=args.batch_size,
        shuffle=shuffle,
        num_workers=args.num_workers,
        drop_last=drop_last,
        collate_fn=collate_fn
    )
    return dataloader0, dataloader1


In [ ]:
#eval.py

from tqdm import tqdm
import torch
import torch.nn.functional as F

from bert_pretrained import extract_features, bert_tokenizer
from utils import covariance, sqrtm


@torch.no_grad()
def _frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):
    """PyTorch implementation of the Frechet Distance.
    The Frechet distance between two multivariate Gaussians X_1 ~ N(mu_1, C_1)
    and X_2 ~ N(mu_2, C_2) is
            d^2 = ||mu_1 - mu_2||^2 + Tr(C_1 + C_2 - 2*sqrt(C_1*C_2)).
    Stable version by Dougal J. Sutherland.
    Params:
    -- mu1   : Numpy array containing the activations of a layer of the
               inception net (like returned by the function 'get_predictions')
               for generated samples.
    -- mu2   : The sample mean over activations, precalculated on an
               representative data set.
    -- sigma1: The covariance matrix over activations for generated samples.
    -- sigma2: The covariance matrix over activations, precalculated on an
               representative data set.
    Returns:
    --   : The Frechet Distance.
    """
    assert mu1.shape == mu2.shape, \
        'Two mean vectors have different lengths'
    assert sigma1.shape == sigma2.shape, \
        'Two covariances have different dimensions'

    diff = mu1 - mu2

    # Product might be almost singular
    # NOTE: the matrix square root is forced to be real
    covmean = sqrtm(sigma1 @ sigma2)
    if not torch.isfinite(covmean).all():
        msg = ('fid calculation produces singular product; '
               'adding %s to diagonal of cov estimates') % eps
        print(msg)
        offset = torch.eye(sigma1.size(0)) * eps
        covmean = sqrtm((sigma1 + offset) @ (sigma2 + offset))

    return (diff @ diff + torch.trace(sigma1) + torch.trace(sigma2)
            - 2 * torch.trace(covmean))


@torch.no_grad()
def calculate_frechet_distance(text1, text2, verbose=False):
    first = []
    for line in tqdm(text1, disable=not verbose):
        first.append(extract_features(line.strip()))
    first = torch.stack(first, dim=0)
    mu1 = torch.mean(first, dim=0)
    sigma1 = covariance(first, rowvar=False)

    second = []
    for line in tqdm(text2, disable=not verbose):
        second.append(extract_features(line.strip()))
    second = torch.stack(second, dim=0)
    mu2 = torch.mean(second, dim=0)
    sigma2 = covariance(second, rowvar=False)

    return _frechet_distance(mu1, sigma1, mu2, sigma2)


@torch.no_grad()
def calculate_accuracy(clf, text, labels, verbose=False):
    outputs = []
    for line in tqdm(text, disable=not verbose):
        inputs = bert_tokenizer(
            line.strip(),
            add_special_tokens=True,
            return_tensors='pt',
            padding=True
        ).to(labels.device)
        outputs.append(clf(**inputs))
    outputs = torch.cat(outputs, dim=0)
    loss = F.cross_entropy(outputs, labels)
    preds = torch.argmax(outputs, dim=1)
    acc = (preds == labels).float().mean()

    return loss, acc


In [ ]:
#model.py
import torch
import torch.nn as nn
from torch.distributions import Categorical
import torch.nn.functional as F


class Encoder(nn.Module):
    def __init__(self, embedding, dim_y, dim_z):
        """
        Required parameters:
            embedding: nn.Embedding
            dim_y: hyperparam
            dim_z: hyperparam

        구성요소:
            Embedding Layer
            Fully connected Layer (to get latent variable `y`)
            unidirectional GRU with layer 1

        """
        super().__init__()
        self.fc = nn.Linear(1, dim_y)  # output: (batch_size, dim_y)
        self.register_buffer("init_z", torch.zeros(dim_z, requires_grad=False))
        self.embed = embedding

        self.rnn = nn.GRU(
            self.embed.embedding_dim,
            dim_y + dim_z,
            num_layers=1
        )
        self.dim_y = dim_y

    def forward(self, labels, src, src_len):
        """
        labels: torch.LongTensor with shape (batch_size,)
        src: torch.LongTensor with shape (max_seq_len, batch_size)
        src_len: list of seq lengths
        """
        labels = labels.unsqueeze(-1)  # (batch_size, 1), 엔코더 밖에서 해줘도 괜찮을 듯
        src = self.embed(src)  # (max_seq_len, batch_size, embed_dim)
        packed_embed = nn.utils.rnn.pack_padded_sequence(  # input to rnn
            src,
            src_len,
            enforce_sorted=False
        )

        # initial hidden state of the encoder: concat (y, z)
        # [ batch size: src.shape[1] , dim_z ]
        init_z = self.init_z.repeat(src.shape[1], 1)
        init_hidden = torch.cat((self.fc(labels), init_z), -1)

        _, hidden = self.rnn(packed_embed, init_hidden.unsqueeze(0))
        # hidden : hidden_state of the final time step
        hidden = hidden.squeeze(0)  # (batch_size, hidden_dim)
        z = hidden[:, self.dim_y:]  # (batch_size, dim_z)
        return z


class Generator(nn.Module):
    def __init__(self, embeddings, dim_y, dim_z, temperature,
                 bos_token_id=8002, use_gumbel=True):
        """
        Required parameters:
            embedding: nn.Embedding()
            dim_y: .
            dim_z: .
            temperature: refer to paper
            bos_token_id: tokenizer.bos_token_id

        Components:
            Fully connected Layer (to get latent `y`)
            Word Embedding
            Unidirectional GRU (layer=1)
            Fully connected Layer (prediction)
        """
        super().__init__()
        self.temperature = temperature
        self.dim_h = dim_y + dim_z

        self.embed = embeddings  # type(embeddings) = nn.Embedding
        self.register_buffer(
            "bos_token_id",
            torch.tensor([bos_token_id], dtype=int)
        )

        self.fc = nn.Linear(1, dim_y)  # latent `y`
        # The hidden state's dimension: dim_y + dim_z
        self.rnn = nn.GRU(self.embed.embedding_dim, self.dim_h, num_layers=1)
        self.fc_out = nn.Linear(self.dim_h, self.embed.num_embeddings)  # prediction
        self.use_gumbel = use_gumbel

    def transfer(self, z, label_to_transfer_to, eos_token_id=8003, max_len=64,
                 top_k=5):
        label = label_to_transfer_to
        assert z.size(0) == 1 and label.size(0) == 1
        label = label.unsqueeze(-1)

        h0 = torch.cat((self.fc(label), z), -1)  # (1, dim_h)

        input = self.embed(self.bos_token_id).repeat(1, 1).unsqueeze(0)  # (1, 1, embed_dim)
        hidden = h0.unsqueeze(0)  # (1, 1, dim_h)

        predictions = []
        for t in range(max_len):
            output, hidden = self.rnn(input, hidden)
            prediction = self.fc_out(output)  # (1, 1, num_embedding)

            top_k_logits = prediction.topk(top_k).values
            top_k_indices = prediction.topk(top_k).indices

            sample = Categorical(logits=top_k_logits).sample().item()
            prediction = top_k_indices[:, :, sample]  # (1, 1)
            predictions.append(prediction.item())
            if prediction == eos_token_id:
                break

            input = self.embed(prediction)  # (1, 1, embed_dim)

        return predictions

    def forward(self, z, labels, src, src_len, transfered=True):
        """
        Required Parameters
            src: original sentence [seq_len, batch_size]
            src_len: original sentence len [batch_size]

        * using gumbel_softmax

        Returns:
            outpus: to feed to discriminator
            predictions: get loss_rec
        """
        labels = labels.unsqueeze(-1)  # (batch_size, 1)

        # placeholders for outputs and prediction tensors
        outputs = []
        predictions = []

        if transfered:
            # Feed previous decoding
            h0 = torch.cat((self.fc(1-labels), z), -1)  # h0_transfered

            input = self.embed(self.bos_token_id).repeat(src.shape[1], 1)
            input = input.unsqueeze(0)      # [1, batch_size, embed_dim]
            hidden = h0.unsqueeze(0)        # [1, batch_size, hidden_size]
            for t in range(1, max(src_len)):
                output, hidden = self.rnn(input, hidden)
                outputs.append(output)
                prediction = self.fc_out(output)
                predictions.append(prediction)

                # 원본코드의 softsample_word를 참조
                T = self.temperature
                if self.use_gumbel:
                    input = torch.matmul(
                        F.gumbel_softmax(prediction / T, dim=-1),
                        self.embed.weight
                    )
                else:
                    input = torch.matmul(
                        F.softmax(prediction / T, dim=-1),
                        self.embed.weight
                    )

        else:
            # Teacher Forcing
            h0 = torch.cat((self.fc(labels), z), -1)  # h0_original
            input = self.embed(src[0]).unsqueeze(0)
            hidden = h0.unsqueeze(0)  # [1, batch_size, hidden_size]
            for t in range(1, max(src_len)):
                output, hidden = self.rnn(input, hidden)
                outputs.append(output)
                prediction = self.fc_out(output)
                predictions.append(prediction)
                input = self.embed(src[t]).unsqueeze(0)

        outputs = torch.cat([h0.unsqueeze(0)] + outputs, 0)  # according to the paper you need h0 in the sequence to feed the discriminator
        predictions = torch.cat(predictions, 0)
        # outputs = [ 1 + max_seq_len, batch_size, hidden_size]
        # predictions = [max_seq_len, batch_size, hidden_size]
        return outputs, predictions


class Discriminator(nn.Module):
    def __init__(self, dim_h, n_filters, filter_sizes, output_dim=1,
                 dropout=0.5):
        super().__init__()
        self.cnn = TextCNN(dim_h, n_filters, filter_sizes, output_dim=1,
                           dropout=dropout)

    def forward(self, h_sequence):
        # h_sequence: [seq_len, batch_size, hidden_dim]
        return self.cnn(h_sequence.transpose(0, 1))


class TextCNN(nn.Module):
    def __init__(self, dim_h, n_filters, filter_sizes, output_dim, dropout):
        # 원본 코드 상의 output_dim은 1
        super().__init__()
        self.convs = nn.ModuleList([
            nn.Conv2d(in_channels=1,
                      out_channels=n_filters,
                      kernel_size=(fs, dim_h))
            for fs in filter_sizes
        ])
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, hiddens):
        # don't forget the permutation
        # hiddens = [batch_size, hiddens seq len, dim_h]
        hiddens = hiddens.unsqueeze(1)
        # hiddens = [batch_size, 1, hiddens seq len, dim_h]

        conved = [
            F.leaky_relu(conv(hiddens)).squeeze(3) for conv in self.convs
        ]
        # conved[n] = [batch size, n_filters, dim_h - filter_sizes[n] + 1]

        pooled = [
            F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved
        ]
        # pooled[n] = [batch size, n_filters]

        cat = self.dropout(torch.cat(pooled, dim=1))
        # cat = [batch size, n_filters * len(filter_sizes)]
        return self.fc(cat)


In [ ]:
#loss.py

import torch
import torch.nn.functional as F


def gradient_penalty(real_data, gen_data, disc):
    """code modified from https://github.com/EmilienDupont/wgan-gp"""
    # Calculate interpolation
    batch_size = real_data.size(1)
    alpha = torch.rand(1, batch_size, 1).to(real_data)
    alpha = alpha.expand_as(real_data)
    interpolated = alpha * real_data.data + (1 - alpha) * gen_data.data
    interpolated.requires_grad = True

    # Calculate probability of interpolated examples
    prob_interpolated = disc(interpolated)

    # Calculate gradients of probabilities with respect to examples
    gradients = torch.autograd.grad(
        outputs=prob_interpolated,
        inputs=interpolated,
        grad_outputs=torch.ones_like(prob_interpolated),
        create_graph=True,
        retain_graph=True
    )
    # Gradients have shape (seq_len, batch_size, hidden_dim)
    # so flatten to easily take norm per example in batch
    gradients = gradients[0].transpose(1, 0).view(batch_size, -1)

    # Derivatives of the gradient close to 0 can cause problems because of
    # the square root, so manually calculate norm and add epsilon
    gradients_norm = torch.sqrt(torch.sum(gradients ** 2, dim=1) + 1e-12)

    # Return gradient penalty
    return ((gradients_norm - 1) ** 2).mean()


def wgan_disc(data, labels):
    probs = torch.sigmoid(data)
    return (-probs * labels + probs * (1 - labels)).mean()


def wgan_gen(data, labels):
    probs = torch.sigmoid(data)
    return (-probs * labels).mean()


def loss_fn(gan_type='vanilla', disc=True):
    if gan_type == 'vanilla':
        return F.binary_cross_entropy_with_logits
    elif gan_type == 'lsgan':
        return F.mse_loss
    elif gan_type == 'wgan-gp':
        return wgan_disc if disc else wgan_gen
    else:
        raise NotImplementedError


In [ ]:
#bert_pretrained_model
import torch
import torch.nn as nn
from transformers import BertModel

from bert_pretrained.tokenizer import bert_tokenizer
from options import args


if args.language == 'ko':
    model_type = 'monologg/kobert'
else:
    model_type = 'bert-base-cased'
BERT = BertModel.from_pretrained(model_type).to(args.device)


def get_bert_word_embedding():
    num_embeddings = (bert_tokenizer.vocab_size
                      + len(bert_tokenizer.get_added_vocab()))
    embed_dim = BERT.embeddings.word_embeddings.embedding_dim

    # need to add embedding for bos and eos token
    embedding = nn.Embedding(
        num_embeddings,
        embed_dim,
        padding_idx=bert_tokenizer.pad_token_id
    )
    embedding.weight.data[:bert_tokenizer.vocab_size].copy_(
        BERT.embeddings.word_embeddings.weight.data
    )
    return embedding


@torch.no_grad()
def extract_features(text):
    inputs = bert_tokenizer(
        text,
        add_special_tokens=True,
        return_tensors='pt',
        padding=True
    ).to(args.device)
    features = BERT(**inputs)[0]
    # sentence embedding = mean of word embedding
    return features.squeeze(0).mean(0)
